## Deliverable 2. Create a Customer Travel Destinations Map.

In [38]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import matplotlib.pyplot as plt
import numpy as np

# Import the random module.
import random
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [51]:
# 1. Import the WeatherPy_database.csv file.
city_data_df = pd.read_csv(parentdir+'\\Weather_Database\\WeatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kovrov,RU,56.36,41.32,54.59,73,100,7.61,overcast clouds
1,1,Ukiah,US,39.15,-123.21,84.20,26,1,6.93,clear sky
2,2,Porbandar,IN,21.64,69.61,80.60,88,20,4.70,haze
3,3,Vasterhaninge,SE,59.12,18.10,60.01,54,33,14.25,scattered clouds
4,4,Qaanaaq,GL,77.48,-69.36,23.23,81,0,8.34,clear sky


In [52]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [53]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kovrov,RU,56.36,41.32,54.59,73,100,7.61,overcast clouds
1,1,Ukiah,US,39.15,-123.21,84.20,26,1,6.93,clear sky
2,2,Porbandar,IN,21.64,69.61,80.60,88,20,4.70,haze
3,3,Vasterhaninge,SE,59.12,18.10,60.01,54,33,14.25,scattered clouds
5,5,Tiksi,RU,71.69,128.87,52.25,72,82,3.04,broken clouds
6,6,Kapaa,US,22.08,-159.32,82.40,65,90,10.29,overcast clouds
7,7,Cape Town,ZA,-33.93,18.42,57.00,93,75,11.41,broken clouds
8,8,Ucluelet,CA,48.93,-125.55,59.00,90,34,8.01,scattered clouds
9,9,Avarua,CK,-21.21,-159.78,78.80,83,1,8.05,light rain
10,10,Waipawa,NZ,-41.41,175.52,60.80,82,41,18.34,scattered clouds


In [57]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [58]:
preferred_cities_df.count()


City_ID                594
City                   594
Country                594
Lat                    594
Lng                    594
Max Temp               594
Humidity               594
Cloudiness             594
Wind Speed             594
Current Description    594
dtype: int64

In [56]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [60]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kovrov,RU,54.59,overcast clouds,56.36,41.32,
1,Ukiah,US,84.20,clear sky,39.15,-123.21,
2,Porbandar,IN,80.60,haze,21.64,69.61,
3,Vasterhaninge,SE,60.01,scattered clouds,59.12,18.10,
5,Tiksi,RU,52.25,broken clouds,71.69,128.87,
6,Kapaa,US,82.40,overcast clouds,22.08,-159.32,
7,Cape Town,ZA,57.00,broken clouds,-33.93,18.42,
8,Ucluelet,CA,59.00,scattered clouds,48.93,-125.55,
9,Avarua,CK,78.80,light rain,-21.21,-159.78,
10,Waipawa,NZ,60.80,scattered clouds,-41.41,175.52,


In [64]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"


    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json() 
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

    
   
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [71]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kovrov,RU,54.59,overcast clouds,56.36,41.32,Hostel Domino
1,Ukiah,US,84.20,clear sky,39.15,-123.21,Hampton Inn Ukiah
2,Porbandar,IN,80.60,haze,21.64,69.61,Lords Inn Porbandar
3,Vasterhaninge,SE,60.01,scattered clouds,59.12,18.10,Viamap
5,Tiksi,RU,52.25,broken clouds,71.69,128.87,Arktika
6,Kapaa,US,82.40,overcast clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
7,Cape Town,ZA,57.00,broken clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
8,Ucluelet,CA,59.00,scattered clouds,48.93,-125.55,Black Rock Oceanfront Resort
9,Avarua,CK,78.80,light rain,-21.21,-159.78,Paradise Inn
10,Waipawa,NZ,60.80,scattered clouds,-41.41,175.52,Tora Coastal Walk


In [84]:
hotel_df["Hotel Name"].replace("",np.nan,inplace=True)

In [80]:
hotel_df[hotel_df["Hotel Name"] == "" ].count()

City                   51
Country                51
Max Temp               51
Current Description    51
Lat                    51
Lng                    51
Hotel Name             51
dtype: int64

In [87]:
hotel_df["Hotel Name"].isnull().sum()

0

In [86]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [88]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [94]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [95]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer) 

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))